# 개요

얼굴 표정을 통한 강아지의 감정 분석 AI 학습을 위한 영상 전처리
 

# 내용

1. 파일 불러오기.
2. 영상에 강아지가 있는지,  1 마리만 있는지 유효성 검사 실행.
3. Grayscale 처리 : RGB는 3차원의 색공간을 가지고 있다. 즉, `R : 0~255, G : 0~255, B : 0~255` 를 가지고 있으며 이를 계산하면 `255x255x255 = 16581374`란 계산을 컴퓨터가 해야하지만.. Grayscale은 `1차원인 0~255`만 따지게 되므로 연산량이 대폭 감소하며 정확도를 올릴 수 있다.
4. Alignment 처리 : 학습 정확도를 올리기 위해 기울어진 얼굴을 수직으로 정렬한다.
5. Crop 처리 : 얼굴 외 필요없는 배경은 잘라낸다.
6. 처리된 영상을 별도로 저장.

 
# 개선 방향
하나의 이미지의 여러개의 강아지 얼굴이 있을시 다중 처리할 수 있도록 개선 예정

----------------

In [1]:
import dlib, cv2, os, sys
from imutils import face_utils
from tkinter import filedialog as fd
from tkinter import messagebox as mb
import matplotlib.pyplot as plt
import tkinter as tk
import numpy as np

## 변수 선언 및 학습 모델 Load
학습모델 출처 : https://github.com/kairess/dog_face_detector

In [2]:
img_origin = 0
img_gray = 0
filename = ''
img_bin = 0
img_canny_edge = 0
dets = []
detector = dlib.cnn_face_detection_model_v1('dogHeadDetector.dat')
predictor = dlib.shape_predictor('landmarkDetector.dat')

## 함수  : 로컬 파일 셀렉트

In [3]:
def file_open():
    global img_origin,filename,dets
    
    filename = fd.askopenfilename()
    print(filename)
    img_origin = cv2.imread(filename)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    dets = detector(img_origin, upsample_num_times=1)

    print('강아지 얼굴 데이터')
    print(dets)
    print('강아지 얼굴 갯수 : ')
    print(len(dets))
    
    if len(dets) != 1 : 
        mb.showinfo(title='', message='사진에 강아지가 0 또는 여러 마리에요. 하나인 사진을 등록해주세요.', icon='info')
    else:
        mb.showinfo(title='', message='등록 완료. 랜드마크를 생성해주세요.', icon='info')

## 함수 : 강아지 얼굴 랜드마크 생성

In [4]:
def create_landmark():
    global img_origin,filename,dets
    
    if type(img_origin) is int:
        mb.showinfo(title='', message='먼저 파일을 선택해주세요.', icon='info')
        
    elif len(dets) == 0:
        mb.showinfo(title='', message='정상적인 강아지 사진이 아닙니다. 다시 사진을 등록해주세요.', icon='info')
        
    else:
        img_landmark = img_origin.copy()

        #### BOX 생성
        for i, d in enumerate(dets):
            print("Detection {}: Left: {} Top: {} Right: {} Bottom: {} Confidence: {}".format(i, d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom(), d.confidence))

            x1, y1 = d.rect.left(), d.rect.top()
            x2, y2 = d.rect.right(), d.rect.bottom()

            cv2.rectangle(img_landmark, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=(255,0,0), lineType=cv2.LINE_AA)

            
        #### 랜드마크 생성
        shapes = []

        for i, d in enumerate(dets):
            shape = predictor(img_origin, d.rect)
            shape = face_utils.shape_to_np(shape)

            for i, p in enumerate(shape):
                shapes.append(shape)
                cv2.circle(img_landmark, center=tuple(p), radius=3, color=(0,0,255), thickness=-1, lineType=cv2.LINE_AA)
                cv2.putText(img_landmark, str(i), tuple(p), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        cv2.imshow(filename,img_landmark)

## UI 생성

In [ ]:
window = tk.Tk()
window.title("assignment")

btn_file_open = tk.Button(window, width=40, padx=5, pady=5, text='File Open...', command=file_open)
btn_file_open.grid(row=0, column=0, sticky=tk.EW)

btn_landmark = tk.Button(window, padx=5, pady=5, text='Create Landmark', command=create_landmark)
btn_landmark.grid(row=1, column=0, sticky=tk.EW)

btn_binarize = tk.Button(window, padx=5, pady=5, text='Binarization')
btn_binarize.grid(row=2, column=0, sticky=tk.EW)

btn_edge = tk.Button(window, padx=5, pady=5, text='Canny Edge')
btn_edge.grid(row=3, column=0, sticky=tk.EW)



window.mainloop()

/Users/righthot/Documents/dev/python-work/py_dog_emotion/img/00001_dog.jpg
강아지 얼굴 데이터
mmod_rectangles[[(117, 25) (410, 318)]]
강아지 얼굴 갯수 : 
1
Detection 0: Left: 117 Top: 25 Right: 410 Bottom: 318 Confidence: 1.0348947048187256
